In [1]:
#import needed package
#---------------------------------------------------------
import pandas as pd
import numpy as np
import sklearn
import autosklearn
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import datetime
import autosklearn.classification
from sklearn.preprocessing import LabelEncoder #OrdinalEncoder

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
#print versions
#---------------------------------------------------------
print('The scikit-learn version is {}.'.format(sklearn.__version__))
print('The Auto scikit-learn version is {}.'.format(autosklearn.__version__))

The scikit-learn version is 0.19.2.
The Auto scikit-learn version is 0.5.2.


In [3]:
#Load Data from CSV
#--------------------------------------------------------
filepath = '/data/home/eissa/Downloads/datasets_91-BayesianNetworkGenerator_mfeat-fourier_small.csv'
rawdata = pd.read_csv( filepath ,  sep=','  ,  low_memory=False )
rawdata.shape

(1000000, 77)

In [4]:
#Categorical feature encoding
#--------------------------------------------------------
label = 'label'
col = [label]
for i in range(76):
    col.append('c' + str(i))
rawdata.columns = col
rawdata.head(10)
le = LabelEncoder()
for column in rawdata: 
    rawdata[column] = le.fit_transform(rawdata[column])

In [5]:
rawdata.head(10)

,label,c0,c1,c2,c3,c4,c5,c6,c7,c8,...,c66,c67,c68,c69,c70,c71,c72,c73,c74,c75
0,2,1,1,1,0,1,1,2,1,0,...,0,0,1,1,1,2,1,2,1,7
1,0,0,0,1,0,0,1,0,1,1,...,0,0,1,1,1,0,1,0,1,8
2,2,2,0,1,0,1,1,2,0,1,...,1,1,0,0,0,0,0,1,1,3
3,1,0,1,0,1,0,1,0,0,0,...,0,0,1,1,0,1,1,0,0,6
4,1,0,1,0,0,1,2,2,0,0,...,0,2,0,0,1,1,0,1,1,9
5,1,1,0,0,0,1,2,0,2,0,...,0,0,0,0,0,1,0,0,0,5
6,0,0,0,0,0,1,2,0,0,0,...,1,0,1,1,0,1,1,0,0,9
7,1,2,0,2,0,1,1,0,0,0,...,1,1,1,2,1,0,0,0,0,1
8,0,1,1,0,2,0,1,1,0,1,...,0,2,1,0,1,0,0,2,0,6
9,2,1,1,0,1,0,1,2,1,0,...,1,1,1,1,1,1,0,0,0,6


In [ ]:
#Label casting
#--------------------------------------------------------
rawdata[[label]] = rawdata[[label]].astype(int)

In [6]:
# split data into Testing and training
# ---------------------------------------------------------------------------------------------------------------------
np.random.seed(1234)
msk = np.random.rand(len(rawdata)) < 0.8
training_data = rawdata[msk]
Testing_data = rawdata[~msk]

X_train = training_data.loc[:, training_data.columns != label]
Y_train = training_data.loc[:, training_data.columns == label]

X_test = Testing_data.loc[:, Testing_data.columns != label]
Y_test = Testing_data.loc[:, Testing_data.columns == label]

In [10]:
# auto ML
# ---------------------------------------------------------------------------------------------------------------------
t1 = datetime.datetime.now()
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=100, 
                                                          #per_run_time_limit=100,
                                                          include_preprocessors=["no_preprocessing"],
                                                          ensemble_memory_limit=45000, 
                                                          seed=1, 
                                                          ml_memory_limit=45000)
automl.fit(X_train, Y_train)
y_hat = automl.predict(X_test)
t2 = datetime.datetime.now()
print("Time:" + str((t2 - t1).total_seconds()))
print("Accuracy score", accuracy_score(Y_test, y_hat))


/anaconda/envs/py35/lib/python3.5/site-packages/autosklearn/automl.py:887: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Will change shape via np.ravel().
  y = self._check_y(y)
/anaconda/envs/py35/lib/python3.5/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)


[WARNING] [2020-01-06 15:00:17,120:EnsembleBuilder(1):d4cbceb057fffedc2cd906fa6a4fe477] No models better than random - using Dummy Score!
[WARNING] [2020-01-06 15:00:17,180:EnsembleBuilder(1):d4cbceb057fffedc2cd906fa6a4fe477] No models better than random - using Dummy Score!
Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (95.663719)
[WARNING] [2020-01-06 15:00:19,189:EnsembleBuilder(1):d4cbceb057fffedc2cd906fa6a4fe477] No models better than random - using Dummy Score!
[WARNING] [2020-01-06 15:00:21,197:EnsembleBuilder(1):d4cbceb057fffedc2cd906fa6a4fe477] No models better than random - using Dummy Score!
[WARNING] [2020-01-06 15:00:23,205:EnsembleBuilder(1):d4cbceb057fffedc2cd906fa6a4fe477] No models better than random - using Dummy Score!
[WARNING] [2020-01-06 15:00:25,213:EnsembleBuilder(1):d4cbceb057fffedc2cd906fa6a4fe477] No models better than random - using Dummy Score!
[WARNING] [2020-01-06 15:00:27,2

In [ ]:
automl.cv_results_

In [ ]:
automl.show_models(

In [ ]:
automl.sprint_statistics()